# Load Modules

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import subprocess
from plotly.subplots import make_subplots
from multiprocessing import Pool
from tqdm import tqdm
import os
import subprocess


pio.templates.default = 'plotly_white'
pd.options.mode.chained_assignment = None

# Load Data

## Define Paths

### Enrichment Set

In [2]:
# Choice of Enriched Set Parameters
enriched_set = "z10_c8"

# Input Enriched CSV
fn_input_csv = "../data/enriched/TororoKanunguRound2/enrichments/{}_bool.csv".format(enriched_set)

# Input Peptide List
fn_input_peptides = "../data/enriched/TororoKanunguRound2/enrichments/{}_peptides.txt".format(enriched_set)

### Peptide Meta Data

In [3]:
# Header to Index Dictionary
fn_dict = "../data/meta/peptide_meta/idx_header_map.tab"

# Index to amino acid sequence
fn_idx_aa = "../data/meta/peptide_meta/target_aa.fa"

# IEDB sourced Falciparum Epitope Database
fn_iedb = "../data/IEDB_falciparum/falciparum_epitopes.fa"

### Directory Paths

In [4]:
# Path where MCD-MotifSearch was cloned
git_dir = os.path.realpath("../src/MCD-MotifSearch")

### Create Results Directory

In [5]:
if not os.path.exists("../results/"):
    os.mkdir("../results")

## Load Data

### Load Peptide Meta

In [6]:
# generic sequence reader
def seqReader(fn):
    """
    iterate through sequences and yield as generator
    """
    def openSeq(fn):
        if 'gz' in fn:
            return gzip.open(fn, 'rt')
        else:
            return open(fn, 'r')

    def num_iter(fn):
        if 'fastq' in fn or 'fq' in fn:
            return 4
        else:
            return 2

    n = num_iter(fn)

    with openSeq(fn) as f:
        while True:
            try:
                yield [next(f).strip('\n') for _ in range(n)]
            except StopIteration:
                break

# creates header to target index lookup table
def loader_header_dict(fn):
    d = {}
    f = open(fn, "r+")
    for line in f:
        idx, header = line.strip().split("\t")
        d[header] = idx
    
    return d
    
# creates target index to amino acid sequence lookup table
def target_index_aa_dict(fn):
    d = {}
    for header, seq in seqReader(fn):
        header = header.strip(">")
        d[header] = seq
    return d
        

header_idx = loader_header_dict(fn_dict)
idx_aa = target_index_aa_dict(fn_idx_aa)

### Load Input Data

In [7]:
# creates a list of peptide indices from header list
def convert_peptide_to_index(fn, header_idx):
    indices = []
    f = open(fn ,"r+")
    for line in f:
        if "#" in line:
            continue
        indices.append(header_idx[line.strip()])
    
    return indices

# loads input matrix with indices as target peptide indices
def load_input_matrix(fn_input_csv, fn_input_peptides, header_idx):
    idx_list = convert_peptide_to_index(fn_input_peptides, header_idx)
    input_matrix = pd.read_csv(fn_input_csv)
    input_matrix.index = idx_list
    input_matrix.name = "peptide"
    return input_matrix


input_matrix = load_input_matrix(fn_input_csv, fn_input_peptides, header_idx)

### Create Necessary Input Fasta

In [8]:
mcd_input_fa = "../data/enriched/TororoKanunguRound2/enrichments/{}_mcd_input.fa".format(enriched_set)
f = open(mcd_input_fa, "w+")
for i in input_matrix.index:
    f.write(">{}\n{}\n".format(i, idx_aa[i]))
f.close() 

# Run MCD-MotifSearch

## Create Linkage Network and Run Minimization

In [9]:
# Run LinkageMinimization
p = subprocess.Popen(
    args = " ".join([
        os.path.join(git_dir, "pipelines/linkage_minimization.sh"),
        git_dir.strip(), 
        mcd_input_fa.strip(),
        enriched_set.strip(),
        "../results/mcdms_enriched/".strip()
    ]),
    stdout = subprocess.PIPE,
    stderr = subprocess.PIPE,
    shell=True
)
stdout, stderr = p.communicate()
for line in stderr.decode("ascii").split("\n"):
    print(line)

Reading graph from ../results/mcdms_enriched//cliques/current.dimacs...OK
Searching for a single maximum size clique...
  2/2514 (max  1)  0.01 s  (0.00 s/round)
259/2514 (max  2)  0.02 s  (0.00 s/round)
473/2514 (max  3)  0.02 s  (0.00 s/round)
641/2514 (max  4)  0.02 s  (0.00 s/round)
798/2514 (max  5)  0.02 s  (0.00 s/round)
925/2514 (max  6)  0.02 s  (0.00 s/round)
1038/2514 (max  7)  0.02 s  (0.00 s/round)
1125/2514 (max  8)  0.02 s  (0.00 s/round)
1198/2514 (max  9)  0.02 s  (0.00 s/round)
1261/2514 (max 10)  0.02 s  (0.00 s/round)
1316/2514 (max 11)  0.02 s  (0.00 s/round)
1367/2514 (max 12)  0.02 s  (0.00 s/round)
1414/2514 (max 13)  0.02 s  (0.00 s/round)
1456/2514 (max 14)  0.02 s  (0.00 s/round)
1491/2514 (max 15)  0.02 s  (0.00 s/round)
1523/2514 (max 16)  0.02 s  (0.00 s/round)
1555/2514 (max 17)  0.02 s  (0.00 s/round)
1582/2514 (max 18)  0.02 s  (0.00 s/round)
1607/2514 (max 19)  0.02 s  (0.00 s/round)
1630/2514 (max 20)  0.02 s  (0.00 s/round)
1652/2514 (max 21)  0.02 s

## Process Cliques with MEME

In [10]:
clique_p = subprocess.Popen(
    args = " ".join([
        os.path.join(git_dir, "pipelines/clique_meme.sh"),
        "../results/mcdms_enriched/".strip(),
        enriched_set
    ]),
    stdout = subprocess.PIPE,
    stderr = subprocess.PIPE,
    shell=True
)
stdout, stderr = clique_p.communicate()
for line in stderr.decode("ascii").split("\n"):
    print(line)

The output directory '../results/mcdms_enriched//motifs/clique_0' already exists.
Its contents will be overwritten.
BACKGROUND: using background model of order 0
PRIMARY (classic): n 442 p0 442 p1 0 p2 0
SEQUENCE GROUP USAGE-- Starts/EM: p0; Trim: p0; pvalue: p0; nsites: p0,p1,p2
SEEDS: maxwords 27404 highwater mark: seq 442 pos 55
BALANCE: samples 442 chars 27404 nodes 1 chars/node 27404
Initializing the motif probability tables for 2 to 1000 sites...
nsites = 1000
Done initializing.

seqs=   442, min_w=  62, max_w=   62, total_size=    27404

motif=1
SEED DEPTHS: 2 4 8 16 32 64 128 256 512 1000
SEED WIDTHS: 7 9 12
starts: w=12, seq=440, l=62				
Starting point: VRKRFYQ
em: w=   7, psites=   2, iter=   0 
Starting point: FIEEMIP
em: w=   7, psites=   4, iter=   0 
Starting point: FIEEMIP
em: w=   7, psites=   8, iter=   0 
Starting point: FIEEMIP
em: w=   7, psites=  16, iter=   0 
Starting point: LVAEVIP
em: w=   7, psites=  32, iter=   0 
Starting point: IVAEVIP
em: w=   7, psites= 

SEED WIDTHS: 7 9 12
starts: w=12, seq=0, l=62				
Starting point: NGNVNGN
em: w=   7, psites=   2, iter=   0 
Starting point: NGNVNGN
em: w=   7, psites=   4, iter=   0 
Starting point: NGNVNGN
em: w=   7, psites=   8, iter=   0 
Starting point: NGNVNGN
em: w=   7, psites=  15, iter=   0 

real	0m0.149s
user	0m0.115s
sys	0m0.026s
The output directory '../results/mcdms_enriched//motifs/clique_159' already exists.
Its contents will be overwritten.
BACKGROUND: using background model of order 0
PRIMARY (classic): n 3 p0 3 p1 0 p2 0
SEQUENCE GROUP USAGE-- Starts/EM: p0; Trim: p0; pvalue: p0; nsites: p0,p1,p2
SEEDS: maxwords 186 highwater mark: seq 3 pos 55
BALANCE: samples 3 chars 186 nodes 1 chars/node 186
Initializing the motif probability tables for 2 to 15 sites...
nsites = 15
Done initializing.

seqs=     3, min_w=  62, max_w=   62, total_size=      186

motif=1
SEED DEPTHS: 2 4 8 15
SEED WIDTHS: 7 9 12
starts: w=12, seq=0, l=62				
Starting point: VAATCDP
em: w=   7, psites=   2, iter

Starting point: EDIIKHNEDVRE
em: w=  12, psites=   4, iter=   0 
Starting point: DIIKHNEDVREE
em: w=  12, psites=   8, iter=   0 
Starting point: EEETEEETEEET
em: w=  12, psites=  16, iter=   0 
Starting point: EEETEEETEEET
em: w=  12, psites=  32, iter=   0 
Starting point: EEDIIKHNEDVR
em: w=  12, psites=  64, iter=   0 
motif=6
SEED DEPTHS: 2 4 8 16 32 64 100
SEED WIDTHS: 7 9 12
starts: w=12, seq=15, l=62				
Starting point: SDIEQIA
em: w=   7, psites=   2, iter=   0 
Starting point: NVAESIV
em: w=   7, psites=   4, iter=   0 
Starting point: EELENEG
em: w=   7, psites=   8, iter=   0 
Starting point: GEIEYIT
em: w=   7, psites=  16, iter=   0 
Starting point: EEELENE
em: w=   7, psites=  32, iter=   0 
Starting point: EEELENE
em: w=   7, psites=  64, iter=   0 
Starting point: EEEIKED
em: w=   7, psites= 100, iter=   0 
Starting point: TDGEIEYIT
em: w=   9, psites=   2, iter=   0 
Starting point: VKNVAESIV
em: w=   9, psites=   4, iter=   0 
Starting point: EIEYITEEE
em: w=   9, ps

Initializing the motif probability tables for 2 to 80 sites...
nsites = 80
Done initializing.

seqs=    16, min_w=  62, max_w=   62, total_size=      992

motif=1
SEED DEPTHS: 2 4 8 16 32 64 80
SEED WIDTHS: 7 9 12
starts: w=12, seq=15, l=62				
Starting point: RNKGSDG
em: w=   7, psites=   2, iter=   0 
Starting point: RNKGSDG
em: w=   7, psites=   4, iter=   0 
Starting point: RNKGSDG
em: w=   7, psites=   8, iter=   0 
Starting point: RNKGSDG
em: w=   7, psites=  16, iter=   0 
Starting point: KGSEGLK
em: w=   7, psites=  32, iter=   0 
Starting point: KNKELLN
em: w=   7, psites=  64, iter=  10 
Starting point: KNKELLN
em: w=   7, psites=  80, iter=  10 
Starting point: LRNKGSDGL
em: w=   9, psites=   2, iter=   0 
Starting point: LRNKGSDGL
em: w=   9, psites=   4, iter=   0 
Starting point: LRNKGSDGL
em: w=   9, psites=   8, iter=   0 
Starting point: LRNKGSDGL
em: w=   9, psites=  16, iter=   0 
Starting point: LRNKGSEGL
em: w=   9, psites=  32, iter=   0 
Starting point: DQQIMDIKS

Starting point: DWWKTNG
em: w=   7, psites=  64, iter=   0 
Starting point: DWWKTNG
em: w=   7, psites= 128, iter=  10 
Starting point: DWWKTNG
em: w=   7, psites= 256, iter=  40 
Starting point: DWWKTNG
em: w=   7, psites= 285, iter=  30 
Starting point: HIWNGMICA
em: w=   9, psites=   2, iter=   0 
Starting point: IWNGMICAL
em: w=   9, psites=   4, iter=   0 
Starting point: IWNGMICAL
em: w=   9, psites=   8, iter=   0 
Starting point: IWNGMICAL
em: w=   9, psites=  16, iter=   0 
Starting point: IWNGMICAL
em: w=   9, psites=  32, iter=   0 
Starting point: IWNGMICAL
em: w=   9, psites=  64, iter=   0 
Starting point: IWNGMICAL
em: w=   9, psites= 128, iter=   0 
Starting point: IWNGMICAL
em: w=   9, psites= 256, iter=   0 
Starting point: WWKTNGEHIWNG
em: w=  12, psites=   2, iter=   0 
Starting point: WWKTNGEHIWNG
em: w=  12, psites=   4, iter=   0 
Starting point: WWKTNGEHIWNG
em: w=  12, psites=   8, iter=   0 
Starting point: WWKTNGEHIWNG
em: w=  12, psites=  16, iter=   0 
Star

em: w=   9, psites=   8, iter=   0 
Starting point: APSVEEIVA
em: w=   9, psites=  16, iter=   0 
Starting point: LSDNLLSNLLGG
em: w=  12, psites=   2, iter=   0 
Starting point: VAPTVEEIVAPT
em: w=  12, psites=   4, iter=   0 
Starting point: VAPTVEEIVAPT
em: w=  12, psites=   8, iter=   0 
Starting point: VAPSVEEIVAPT
em: w=  12, psites=  16, iter=   0 
motif=2
SEED DEPTHS: 2 4 8 16 30
SEED WIDTHS: 7 9 12
starts: w=12, seq=5, l=62				
Starting point: LSNLLGG
em: w=   7, psites=   2, iter=   0 
Starting point: SVAENVA
em: w=   7, psites=   4, iter=   0 
Starting point: ESVAENV
em: w=   7, psites=   8, iter=   0 
Starting point: ESVAENV
em: w=   7, psites=  16, iter=   0 
Starting point: ESVAENV
em: w=   7, psites=  30, iter=  10 
Starting point: LLSNLLGGI
em: w=   9, psites=   2, iter=   0 
Starting point: SIASSVDES
em: w=   9, psites=   4, iter=   0 
Starting point: ETEEIKDSI
em: w=   9, psites=   8, iter=   0 
Starting point: ESIASSVDE
em: w=   9, psites=  16, iter=   0 
Starting po

### BLAST Motifs against Epitope DB

In [11]:
blast_p = subprocess.Popen(
    args = " ".join([
        os.path.join(git_dir, "src/clique_analysis/blast_motifs.sh"),
        "../results/mcdms_enriched/motifs/merged_motifs.fa",
        fn_iedb,
        "../results/mcdms_enriched/motifs/motifs.blast.tab"
    ]),
    stdout = subprocess.PIPE,
    stderr = subprocess.PIPE,
    shell=True
)
stdout, stderr = blast_p.communicate()
for line in stderr.decode("ascii").split("\n"):
    print(line)


real	0m0.814s
user	0m0.804s
sys	0m0.007s

